In [18]:
import numpy as np
import math
import random
import copy
import pdb

## class Node

In [19]:
class Node:
  def __init__(self, node_type ,value, successors=None,children=None, name=None, parent=None):
    self.node_type = node_type #function or terminal
    self.value = value #for function its actual func, for terminal its the variable name or the int constant
    self.successors = successors if successors is not None else [] #list of all the successors
    self.children = children if children is not None else [] #list of the children
    self.name = name #name of the function
    self.parent = parent #parent of the node
    self.update_parent()
    if node_type == 'function':
      self.update_successors()

  def update_parent (self):
     for child in self.children:
        child.parent = self

 #evaluates the expression represented by the tree - recursive function
  def evaluate(self, inputs):
    if self.node_type == 'terminal':
      if isinstance(self.value, str):  # variable
        feature_index = int(self.value[1:])
        return inputs[:, feature_index]
      else:
        return np.full(inputs.shape[0], self.value)
    elif self.node_type == 'function':
        evaluated_children = [child.evaluate(inputs) for child in self.children]
        return self.value(*evaluated_children)
    else:
      raise ValueError(f"Invalid node type:{self.node_type}")

  def __str__(self):
    return self.output_function()

  #deep copy to the node and it's successors
  def deep_copy(self):
    copied_children = [child.deep_copy() for child in self.children]
    copied_node = Node(node_type=self.node_type, value=self.value, children=copied_children, name=self.name, parent=None)
    return copied_node

  def get_all_nodes(self):
    return [self] + self.successors

  #get the depth of a tree
  def get_depth(self):
    if self.node_type == 'terminal':
      return 1
    else:
      return 1 + max(child.get_depth() for child in self.children)


  def update_successors(self):
    for child in self.children:
      self.successors.append(child)
      self.successors.extend(child.successors)


 #returns the formula as the given format
  def output_function(self):
    if self.node_type == 'terminal':
      if isinstance(self.value, str):  # variable
        return f"x[{self.value[1:]}]"
      else:  # constant
        return f"{self.value}"
    elif self.node_type == 'function':
      if len(self.children) == 2:  # Binary operation
        left = self.children[0].output_function()
        right = self.children[1].output_function()
        operators_symbols = {'add': '+', 'sub': '-', 'mul': '*'}
        operations_np = {'add': 'np.add', 'sub': 'np.subtract', 'mul': 'np.multiply'}
        if self.name in operators_symbols:
          oper = operators_symbols[self.name]
          return f"({left} {oper} {right})"
        else:
          raise ValueError(f"Unknown binary operation: {self.name}")

      elif len(self.children) == 1:  # Unary operation
        operators = {'sin': 'np.sin', 'cos': 'np.cos', 'pow2': 'np.power', 'pow4': 'np.power', 'pow3' : 'np.pow3', 'arctan' : 'np.arctan'}
        child = self.children[0].output_function()
        if self.name in operators:
          oper = operators[self.name]
          if self.name == 'pow2':
            return f"{oper}({child}, 2)"
          elif self.name == 'pow3':
            return f"{oper}({child}, 3)"
          elif self.name == 'pow4':
            return f"{oper}({child}, 4)"
          else:
            return f"{oper}({child})"
          return f"np.{self.value}({child_expr})"
      else:
        raise ValueError(f"Unknown unary operation: {self.name}")



## Helper Functions

In [20]:
def copy_individual(individual):
  copied_tree = individual['tree'].deep_copy()
  copied_individual = {'tree': copied_tree, 'fitness': individual['fitness']}
  return copied_individual


#evaluate the fitness of a tree
def evaluate_fitness(individual, x_train, y_train):
  tree = individual['tree']
  try:
    y_predictions = tree.evaluate(x_train) #send all x_train together
    if not np.all(np.isfinite(y_predictions)):
      #print('go into infinity')
      return float('inf')
    MSE = np.mean((y_train - y_predictions) ** 2)
  except Exception as e:
    print(e)
    print("evaluate_fitness: Error during evaluation")
    return float('inf')
  return MSE



## Initialization Functions

In [48]:
#all the possible operations
def create_operations():
  unary_operations = {
    'sin': lambda x: np.sin(x),          # sin(a)
    'cos': lambda x: np.cos(x),          # cos(a)
    'pow2': lambda x: np.power(x,2),      # a^2
    'pow4': lambda x: np.power(x,4),      #a^4
    'pow3': lambda x: np.power(x,3),      #a^3
    'arctan': lambda x: np.arctan(x)      #arctan(x)
  }

  binary_operations={
    'add': lambda x,y: np.add(x,y),          # a + b
    'sub': lambda x,y: np.subtract(x,y),     # a - b
    'mul': lambda x,y: np.multiply(x,y),     # a * b
  }
  return binary_operations,unary_operations

#create random tree using full method
def build_full_tree (initial_depth, current_depth, binary_dict, unary_dict, terminals, file_name, random_num):
  if initial_depth == current_depth:
    if random.random() < 0.5: #50% it will be variable and 50% it will be random constant number
      return Node(node_type='terminal',value=random.sample(list(terminals),1)[0])
    else:
      return Node(node_type='terminal',value=random.uniform(-random_num, random_num))
  else:
    oper_type = random.choice(['binary' , 'unary']) #two options
    if oper_type == 'binary':
      func_name ,function = random.choice(list(binary_dict.items()))
      child1 = build_full_tree(initial_depth , current_depth + 1 ,binary_dict ,unary_dict, terminals,file_name, random_num) #two recursive call - operation with two argomants
      child2 = build_full_tree(initial_depth , current_depth + 1 ,binary_dict ,unary_dict, terminals,file_name, random_num)
      node = Node(node_type='function', value=function, children=[child1,child2], name=func_name)
      return node
    elif oper_type == 'unary':
      func_name ,function = random.choice(list(unary_dict.items()))
      child = build_full_tree(initial_depth , current_depth + 1 , binary_dict ,unary_dict ,terminals,file_name, random_num) #one recursive call- operation with one argoment
      node = Node(node_type='function', value=function, children=[child], name=func_name)
      return node

#create random tree using grow method
def build_grow_tree(initial_depth, current_depth, binary_dict, unary_dict , terminals, random_num):
  if initial_depth == current_depth or random.random() < 0.2: #20% to terminate the tree
    if random.random() < 0.5: #50% it will be variable and 50% it will be random constant number
      return Node('terminal',random.sample(list(terminals) ,1)[0])
    else:
      return Node('terminal', random.uniform(-random_num, random_num))
  else:
    oper_type = random.choice(['binary' , 'unary' , 'terminal']) #three options
    if oper_type == 'binary':
      func_name ,function = random.choice(list(binary_dict.items()))
      child1 = build_grow_tree(initial_depth , current_depth + 1, binary_dict, unary_dict, terminals, random_num) #two recursive call - operation with two argomants
      child2 = build_grow_tree(initial_depth , current_depth + 1, binary_dict, unary_dict, terminals, random_num)
      node = Node(node_type='function', value=function, children=[child1,child2], name=func_name)
      return node
    elif oper_type == 'unary':
      func_name ,function = random.choice(list(unary_dict.items()))
      child = build_grow_tree(initial_depth , current_depth + 1, binary_dict,unary_dict, terminals, random_num) #one recursive call- operation with one argoment
      node = Node(node_type='function', value=function, children=[child], name=func_name)
      return node
    else: #oper_type == 'terminal'
      if random.random() < 0.5: #50% it will be variable and 50% it will be random constant number
        return Node('terminal', random.sample(list(terminals), 1)[0])
      else:
        return Node('terminal', random.uniform(-random_num, random_num))

#initialize population using ramped half-and-half
def initialize_population(population_size, initial_depth, binary, unary, terminals, x_train, y_train, file_name, random_num):
  population = []
  for i in range(population_size // 2):
    tree = build_full_tree(initial_depth,current_depth=0,binary_dict=binary,unary_dict=unary, terminals=terminals, file_name=file_name, random_num=random_num)
    individual1 = {'tree': tree, 'fitness': None}
    individual1['fitness'] = evaluate_fitness(individual1, x_train, y_train)
    population.append(individual1)
  for j in range(population_size // 2):
    tree = build_grow_tree(initial_depth,current_depth=0,binary_dict=binary,unary_dict=unary, terminals=terminals, random_num=random_num)
    individual2 = {'tree': tree, 'fitness': None}
    individual2['fitness'] = evaluate_fitness(individual2, x_train, y_train)
    population.append(individual2)
  return population

#get inputs from the given data
def get_inputs(file_name):
  problem = np.load(file_name)
  x_input = problem['x']
  print(f"Num of variables: {x_input.shape[0]}")
  y_input = problem['y']
  x_train = np.transpose(x_input)
  y_train = np.transpose(y_input)
  print(y_train.shape)
  return x_train, y_train

def build_terminals(x_train):
  terminals_set = set()
  for i in range(x_train.shape[1]):
    terminals_set.add(f'x{i}')
  return terminals_set



In [22]:

def tournament_selection(population, tournament_size):
  select_parents = np.random.choice(len(population), tournament_size, replace=False)
  tournament = [population[i] for i in select_parents]
  if random.random() < 0.5: #50% choose the shortest, 50% choose the best
    tournament.sort(key = lambda individual: individual['tree'].get_depth())
  else:
    tournament.sort(key = lambda individual: individual['fitness'], reverse=True)
  return tournament[0], tournament[1]


def crossover(individual1, individual2, max_depth, x_train, y_train):
  tree1 = individual1['tree'].deep_copy()
  tree2 = individual2['tree'].deep_copy()
  all_nodes1 = tree1.get_all_nodes()
  all_nodes2 = tree2.get_all_nodes()
  subtree1 = random.choice(all_nodes1) #select random points for crossover from the root and the successors
  subtree2 = random.choice(all_nodes2)

  if subtree1.parent is not None: #if subtree1 is not the root
    parent1_depth = subtree1.parent.get_depth()
    if parent1_depth + subtree2.get_depth() > max_depth+1: #if the offspring will exceed the max depth dont do crossover
      return individual1, individual2

  if subtree2.parent is not None: #if subtree2 is not the root
    parent2_depth = subtree2.parent.get_depth()
    if parent2_depth + subtree1.get_depth() > max_depth+1:
      return individual1, individual2

  if subtree1.parent is None: #if the first random node is the root
    offspring1 = {'tree': subtree1.deep_copy(), 'fitness': None}
  else:
    parent_subtree1 = subtree1.parent
    index1 = parent_subtree1.children.index(subtree1)
    new_subtree1 = subtree2.deep_copy()
    parent_subtree1.children[index1] = new_subtree1
    new_subtree1.parent = parent_subtree1 #ubdae the parent of the new sub
    parent_subtree1.update_successors()
    offspring1 = {'tree': tree1, 'fitness': None}

  if subtree2.parent is None: #if the second random node is the root
    offspring2 = {'tree': subtree2.deep_copy(), 'fitness': None} #TODO- think if the node is root and we want to replace it
  else:
    parent_subtree2 = subtree2.parent
    index2 = parent_subtree2.children.index(subtree2)
    new_subtree2 = subtree1.deep_copy()
    parent_subtree2.children[index2] = new_subtree2
    new_subtree2.parent = parent_subtree2
    offspring2 = {'tree': tree2, 'fitness': None}
    parent_subtree2.update_successors()

  offspring1['fitness'] = evaluate_fitness(offspring1, x_train, y_train)
  offspring2['fitness'] = evaluate_fitness(offspring2, x_train, y_train)
  return offspring1, offspring2



In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Mutations

In [44]:
#choose a random node and change it to other operation or other terminal
def point_mutation(individual, binary_dict, unary_dict, terminals, x_train, y_train, random_num):
  copied_tree = individual['tree'].deep_copy()
  random_node = random.choice(copied_tree.get_all_nodes())
  if random_node.node_type == 'terminal':
    possible_terminals = terminals - {random_node.value}
    if len(possible_terminals) == 0 : #the set is empty because there is only one variable
      random_node.value = random.uniform(-random_num, random_num) #change the only variable to a costant
    else: #the set is not empty
      if random.random() < 0.5: #50% it will be variable and 50% it will be random constant number
        random_node.value = random.choice(list(possible_terminals))
      else:
        random_node.value = random.uniform(-random_num, random_num)
  elif len(random_node.children) == 2: #operation with two arguments
    possible_operations = set(binary_dict) - {random_node.name}
    random_node.name = random.choice(list(possible_operations))
    random_node.value = binary_dict[random_node.name]
  elif len(random_node.children) == 1: #operation with one argument
      possible_operations = set(unary_dict) - {random_node.name}
      random_node.name = random.choice(list(possible_operations))
      random_node.value = unary_dict[random_node.name]

  random_node.update_successors()
  new_individual = {'tree': copied_tree, 'fitness': None}
  new_individual['fitness'] = evaluate_fitness(new_individual, x_train, y_train)
  return new_individual

#choose a random node and expand it with operation and new random children depends on the operation
def expansion_mutation(individual, binary_dict, unary_dict, terminals, x_train, y_train, max_depth, random_num):
  if individual['tree'].children == []:
    return point_mutation(individual, binary_dict, unary_dict, terminals, x_train, y_train, random_num)
  copied_tree = individual['tree'].deep_copy()
  leaf_nodes = [node for node in copied_tree.get_all_nodes() if node.node_type == 'terminal'] #choose a random leaf node
  random_leaf = random.choice(leaf_nodes)
  possible_operations = list(binary_dict) + list(unary_dict)
  random_func_name = random.choice(possible_operations)
  if random_func_name in binary_dict:
    new_func = binary_dict[random_func_name]
    arity=2
  else:
    new_func = unary_dict[random_func_name]
    arity=1
  new_children = []
  for i in range(arity): #create the new children randomly
    if random.random() < 0.5:
      child= Node(node_type='terminal', value=random.sample(list(terminals),1)[0])
    else:
      child= Node(node_type='terminal', value=random.uniform(-random_num, random_num))
    new_children.append(child)

  new_node = Node(node_type='function', value=new_func, children=new_children, name=random_func_name)

  parent = random_leaf.parent
  index = parent.children.index(random_leaf)
  parent.children[index] = new_node #change the random node to the new node

  if copied_tree.get_depth() > max_depth:
    return point_mutation(individual, binary_dict, unary_dict, terminals, x_train, y_train, random_num)

  new_node.parent = parent
  new_node.update_successors() #TODO- make sure the parent is updated
  new_individual = {'tree': copied_tree, 'fitness': None}
  new_individual['fitness'] = evaluate_fitness(new_individual, x_train, y_train)
  return new_individual


Adjustments

In [45]:
def adjust_rates(best_fitness_history, improvement_bound, xover_prob, mutation_prob, point_prob, elites_rate,
                tournament_size, population_size, max_depth, xover_change, mutation_change, elitism_change):

    tournament_change = 1
    depth_change = 1

    improvement = best_fitness_history[0] - best_fitness_history[-1]

    if improvement < improvement_bound:  # Encourage exploration
        mutation_prob = min(mutation_prob + mutation_change, 0.9)  # upper bound
        elites_rate = max(elites_rate - elitism_change, 0.03)      # Lower bound for elitism rate
        tournament_size = max(tournament_size - tournament_change, 3)  # Lower bound
        max_depth = min(max_depth + depth_change, 8)             # Upper bound
        xover_prob = min(xover_prob + xover_change, 0.9)         # Upper bound
        point_prob = max(point_prob - 0.02, 0.1)                # Lower bound
    else:  # Encourage exploitation
        mutation_prob = min(mutation_prob + mutation_change, 0.9)  # Upper bound
        elites_rate = min(elites_rate + elitism_change, 0.5)       # Upper bound
        tournament_size = min(tournament_size + tournament_change, population_size // 2)  # Prevent exceeding half the population
        xover_prob = max(xover_prob - xover_change, 0.01)         # Lower bound
        point_prob = min(point_prob + 0.02, 0.9)                 # Upper bound

    print(f"Rates are - Crossover: {xover_prob:.2f}, Mutation: {mutation_prob:.2f}, "
          f"Point Mutation: {point_prob:.2f}, Elitism: {elites_rate:.2f}, "
          f"Tournament Size: {tournament_size}, Max Depth: {max_depth}")

    return max_depth, xover_prob, mutation_prob, point_prob, elites_rate, tournament_size


## Main Algorithm

In [52]:
def evolution_step(population, xover_prob, mutation_prob, point_prob, tournament_size, generations, elites_rate, population_size,
                   binary, unary, terminals_set, max_depth, adjust_window, improvement_limit, epsilon, x_train, y_train, xover_change,
                   mutation_change,elitism_change, random_num):

  best_fitness_history = []

  for generation in range(generations):
    new_population = []
    sorted_population = sorted(population, key=lambda Individual: Individual['fitness'])
    min_fitness = sorted_population[0]['fitness']

    if min_fitness <= epsilon: #if the approximation is very close, end the loop and return the best
      break

    best_fitness_history.append(min_fitness)
    if len(best_fitness_history) > adjust_window:
      best_fitness_history.pop(0)  # Maintain the window

    if (generation - 1) % 100 == 0:
      print(f"\nGeneration {generation-1}:")
      print(f"\nMin MSE = {min_fitness}")
      print(sorted_population[0]['tree'].get_depth())

    if (generation - 1) % 10 == 0:
      print(sorted_population[0]['tree'].output_function())

    elites = sorted_population[:int(elites_rate * population_size)] #add the elites_rate of the population to the new population
    new_population.extend([copy_individual(elite) for elite in elites])


    while len(new_population) < len(population): #create the rest of the population
      parent1, parent2 = tournament_selection(population, tournament_size) #select the parents
      if random.random() < xover_prob:
        offspring1, offspring2 = crossover(parent1, parent2, max_depth, x_train, y_train)
      else:
        offspring1, offspring2 = copy_individual(parent1), copy_individual(parent2)

      if random.random() < mutation_prob: #mutate offspring1
        if random.random() < point_prob:
          offspring1 = point_mutation(offspring1, binary, unary, terminals_set, x_train, y_train, random_num)
        else:
          offspring1 = expansion_mutation(offspring1, binary, unary, terminals_set, x_train, y_train, max_depth, random_num)

      if random.random() < mutation_prob: #mutate offspring2
        if random.random() < point_prob:
          offspring2 = point_mutation(offspring2, binary, unary, terminals_set, x_train, y_train, random_num)
        else:
          offspring2 = expansion_mutation(offspring2, binary, unary, terminals_set, x_train, y_train, max_depth, random_num)

      new_population.append(offspring1)
      new_population.append(offspring2)

    population = new_population

    #every adjust_window generations, change the rates accord the improvement
    if (generation - 1) % adjust_window == 0:
      max_depth, xover_prob, mutation_prob,point_prob, elites_rate, tournament_size = adjust_rates(best_fitness_history, improvement_limit, xover_prob, mutation_prob,point_prob, elites_rate, tournament_size, population_size, max_depth, xover_change, mutation_change,elitism_change)

  best_individual = min(population, key=lambda ind: ind['fitness'])
  return population, best_individual, best_fitness_history

In [12]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


operate the main algorithm

In [47]:
def initialize(file_name, generations, population_size, adjust_window, max_depth, initial_depth, elites_rate, tournament_size, xover_prob,
               mutation_prob,point_prob, improvement_limit, xover_change, mutation_change,elitism_change, random_num):
  binary, unary  = create_operations()
  x_train, y_train = get_inputs(file_name)
  terminals_set = build_terminals(x_train)
  epsilon = 1e-40
  population = initialize_population(population_size, initial_depth, binary, unary, terminals_set, x_train, y_train, file_name, random_num)
  population_n , best, history = evolution_step(population, xover_prob, mutation_prob, point_prob, tournament_size, generations, elites_rate, population_size, binary, unary, terminals_set, max_depth, adjust_window, improvement_limit, epsilon, x_train, y_train, xover_change, mutation_change,elitism_change, random_num)
  return population_n , best, history




# case 1

In [ ]:
population , best, history = initialize(file_name = 'problem_1.npz', generations = 1000, population_size = 100, adjust_window = 100, max_depth = 6,
                      initial_depth = 3, elites_rate = 0.05, tournament_size = 10, xover_prob = 0.7, mutation_prob = 0.1, improvement_limit=1e-6,
                      xover_change, mutation_change,elitism_change, random_num=10)

print(f"func1:{best['tree'].output_function()}")
print(f"MSE BEST: {best['fitness']}")

#func1: np.sin(x[0])
#MSE BEST: 3.921193366772412e-34

# case 2

In [ ]:
population , best, history = initialize(file_name= 'problem_2.npz', generations =10000, population_size = 300, adjust_window = 1000, max_depth = 4,
                      initial_depth = 3, elites_rate = 0.05, tournament_size = 8, xover_prob = 0.8, mutation_prob = 0.3, point_prob=0.1,
                      improvement_limit=1000, xover_change=0.02, mutation_change=0.02, elitism_change=0.02, random_num=10)

print(f"func2: {best['tree'].output_function()}")
print(f"MSE: {best['fitness']}")

#func2: ((np.power(-32.49387304054636, 4) * x[0]) + (x[2] + (np.power((((x[0] * x[2]) + ((x[0] * x[2]) + ((x[0] * x[2]) + -96.46096667075665))) * -6.310140402151248), 2) * x[1])))
#MSE: 16636405072324.762


#func2: (np.power(-35.77865211740968, 4) * (np.cos(((x[1] + x[2]) * (0.049825662719513275 * x[0]))) * x[0]))
#MSE: 14712336223303.713


# case 3

In [ ]:
population , best, history = initialize(file_name = 'problem_3.npz', generations = 10000, population_size = 200, adjust_window = 50, max_depth = 6,
                      initial_depth = 3, elites_rate = 0.05, tournament_size = 8, xover_prob = 0.8, mutation_prob = 0.1, point_prob=0.2,
                      improvement_limit= 0.1, xover_change=0.05, mutation_change=0.02, elitism_change=0.02, random_num=10)

print(f"func3: {best['tree'].output_function()}")
print(f"MSE: {best['fitness']}")

#func3: ((np.power((x[2] + (-0.15334261790093606 + -1.3457410645123407)), 2) + np.power((-1.1840450316899656 * x[0]), 2)) + ((x[2] - (np.power(x[1], 2) + x[2])) * x[1]))
#MSE: 79.44187676029182


#func3: ((np.power(x[0], 2) + np.power((-1.1840450316899656 * x[0]), 2)) + ((x[2] - (np.power(x[1], 2) + x[2])) * x[1]))
#MSE: 110.36941364068991

#xover_prob=0.6, mutation_prob=0.9, point_prob=0.5, tournament_size=10,
          #                      generations=10000, elites_rate=0.05, population_size=len(population), binary=binary, unary=unary,
           #                     terminals_set=terminals_set, max_depth=6, adjust_window=100, improvement_limit=10, epsilon=0.000001,
            #                    x_train=x_train, y_train=y_train, xover_change=0.002, mutation_change=0.002,elitism_change=0.02, random_num=10




# case 4

In [ ]:

population , best, history = initialize(file_name= 'problem_4.npz', generations = 2000, population_size = 300, adjust_window = 50, max_depth = 6,
                      initial_depth = 3, elites_rate = 0.05, tournament_size = 8, xover_prob = 0.75, mutation_prob = 0.2, improvement_limit=0.001,
                      xover_change=0.005, mutation_change=0.002, elitism_change=0.02, random_num=10)

func4 = best['tree'].output_function()
print(f"MSE: {best['fitness']}")
print(history)

#((np.cos(x[1]) * 3.167794160296859) + ((np.cos(x[1]) * 3.167794160296859) + 2.6422928296213577))
#MSE: 0.5261675540967335
#generations = 2000, population_size = 300, adjust_window = 50, max_depth = 6,
                   #   initial_depth = 3, elites_rate = 0.05, tournament_size = 8, xover_prob = 0.75, mutation_prob = 0.2, improvement_limit=0.001,
                    #                    xover_change=0.005, mutation_change=0.002, elitism_change=0.02


# case 5

In [ ]:

population , best, history = initialize(file_name= 'problem_5.npz', generations = 1000, population_size = 100, adjust_window = 100, max_depth = 6,
                      initial_depth = 3, elites_rate = 0.05, tournament_size = 5, xover_prob = 0.3, mutation_prob = 0.99,point_prob=0.9,
                      improvement_limit=1e-10, xover_change=0.05, mutation_change=0.02, elitism_change=0.02, random_num=100)

print(f"func5: {best['tree'].output_function()}")
print(f"MSE: {best['fitness']}")


#func5: (x[1] - x[1])
#MSE: 5.572810232617333e-18


# case 6

In [ ]:

population , best, history = initialize(file_name= 'problem_6.npz', generations =100000, population_size = 100, adjust_window = 1000, max_depth = 4,
                      initial_depth = 3, elites_rate = 0.05, tournament_size = 5, xover_prob = 0.7, mutation_prob = 0.4 ,point_prob=0.3,
                      improvement_limit=0.01,xover_change=0.05, mutation_change=0.05, elitism_change=0.02, random_num=10)
print(f"func6: {best['tree'].output_function()}")
print(f"MSE: {best['fitness']}")

#func6: (x[1] - (x[0] - (0.7357051974547986 + x[1])))
#MSE: 0.786963173633151
#generations =10000, population_size = 100, adjust_window = 100, max_depth = 3,
                    #  initial_depth = 3, elites_rate = 0.05, tournament_size = 5, xover_prob = 0.7, mutation_prob = 0.4 ,point_prob=0.3,
                     # improvement_limit=0.01,xover_change=0.05, mutation_change=0.05, elitism_change=0.02

#func6: ((-0.585796320561327 * x[0]) + (x[1] + x[1]))
#MSE: 0.5208720349731156

#generations =100000, population_size = 100, adjust_window = 1000, max_depth = 4,
 #                     initial_depth = 3, elites_rate = 0.05, tournament_size = 5, xover_prob = 0.7, mutation_prob = 0.4 ,point_prob=0.3,
  #                    improvement_limit=0.01,xover_change=0.05, mutation_change=0.05, elitism_change=0.02

# case 7

In [ ]:

population , best, history = initialize(file_name= 'problem_7.npz', generations = 10000, population_size = 300, adjust_window = 500, max_depth = 5,
                      initial_depth = 3, elites_rate = 0.05, tournament_size = 8, xover_prob = 0.8, mutation_prob = 0.3, point_prob=0.3,
                      improvement_limit=0.001,xover_change=0.02, mutation_change=0.05, elitism_change=0.02, random_num=1000)
print(f"func7: {best['tree'].output_function()}")
print(f"MSE: {best['fitness']}")

#func7: np.power(((x[1] * (x[0] * (x[1] + x[0]))) * np.sin(x[0])), 2)
#MSE: 356.2540440349073


#func7: (np.cos((x[0] + np.power(x[1], 2))) + (np.power((x[0] + x[1]), 4) * np.cos(np.cos((x[0] + x[1])))))
#MSE: 373.50928642741917


# case 8

In [ ]:
population , best, history = initialize(file_name= 'problem_8.npz', generations = 1000, population_size = 100, adjust_window = 100, max_depth = 8,
                      initial_depth = 3, elites_rate = 0.05, tournament_size = 10, xover_prob = 0.7, mutation_prob = 0.1, point_prob=0.3,
                      improvement_limit=0.001,xover_change=0.02, mutation_change=0.05, elitism_change=0.02, random_num=1000)
print(f"func8: {best['tree'].output_function()}")
print(f"MSE: {best['fitness']}")

#func8: np.power((np.cos((np.power(np.arctan(x[5]), 3) + -5.240068819211974)) + (4.647572411669573 * x[5])), 3)




In [55]:
def f1(x: np.ndarray) -> np.ndarray:
  return np.sin(x[0])


def f2(x: np.ndarray) -> np.ndarray:
  return (np.power(-35.77865211740968, 4) * (np.cos(((x[1] + x[2]) * (0.049825662719513275 * x[0]))) * x[0]))


def f3(x: np.ndarray) -> np.ndarray:
  return ((np.power((x[2] + (-0.15334261790093606 + -1.3457410645123407)), 2) + np.power((-1.1840450316899656 * x[0]), 2)) + ((x[2] - (np.power(x[1], 2) + x[2])) * x[1]))


def f4(x: np.ndarray) -> np.ndarray:
  return ((np.cos(x[1]) * 3.167794160296859) + ((np.cos(x[1]) * 3.167794160296859) + 2.6422928296213577))


def f5(x: np.ndarray) -> np.ndarray:
  return (x[1] - x[1])


def f6(x: np.ndarray) -> np.ndarray:
  return ((-0.585796320561327 * x[0]) + (x[1] + x[1]))


def f7(x: np.ndarray) -> np.ndarray:
  return np.power(((x[1] * (x[0] * (x[1] + x[0]))) * np.sin(x[0])), 2)


def f8(x: np.ndarray) -> np.ndarray:
  return np.power((np.cos((np.power(np.arctan(x[5]), 3) + -5.240068819211974)) + (4.647572411669573 * x[5])), 3)

